In [2]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai neo4j

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Running Cypher Queries in Neo4j

In this section, we will go through how to write and execute Cypher queries to interact with a Neo4j database. Cypher is a declarative graph query language used to interact with Neo4j databases.

In [ ]:
from neo4j import GraphDatabase


uri = "bolt://localhost:7687"
user = "neo4j"
password = "new_password"  
db_name = "STACDBs"

# Connect to the Neo4j instance
driver = GraphDatabase.driver(uri, auth=(user, password))

def create_database(driver, db_name):
    with driver.session() as session:
        try:
            result = session.run("SHOW DATABASES")
            databases = [record["name"] for record in result]
            
            if db_name not in databases:
                session.run(f"CREATE DATABASE {db_name}")
                print(f"Database {db_name} created successfully.")
            else:
                print(f"Database {db_name} already exists.")
        except Exception as e:
            print(f"Error while creating database: {e}")

# Execute Cypher script
def run_cypher_script(driver, db_name, script_path):
    with driver.session(database=db_name) as session:
        try:
            with open(script_path, 'r') as file:
                cypher_query = file.read()  
                session.run(cypher_query)  
            print("Cypher script executed successfully.")
        except Exception as e:
            print(f"Error executing Cypher script: {e}")


create_database(driver, db_name) 
run_cypher_script(driver, db_name, "STAC.cypher") 

Database STACDBs created successfully.
Cypher script executed successfully.


## Step 1: Connect to the Neo4j Database

To begin working with Neo4j, we need to connect to the Neo4j instance. Make sure that Neo4j is running and that you have the correct credentials.

Here’s an example of how to connect to the database using Python's `neo4j` driver:

In [83]:
def run_cypher_script(driver, db_name, script_path):
    with driver.session(database=db_name) as session:
        try:
            with open(script_path, 'r') as file:
                cypher_query = file.read()  
                session.run(cypher_query)  
            print("Cypher script executed successfully.")
        except Exception as e:
            print(f"Error executing Cypher script: {e}")

try:
    driver = GraphDatabase.driver(uri, auth=(user, password))

    with driver.session() as session:
        result = session.run("SHOW DATABASES")
        print("Databases under the Neo4j instance:")
        for record in result:
            print(f"Database: {record['name']} - Current Status: {record['currentStatus']} - Status Message: {record['statusMessage']}")

except Exception as e:
    print(f"Error connecting to Neo4j: {e}")
finally:
    driver.close()


Databases under the Neo4j instance:
Database: neo4j - Current Status: online - Status Message: 
Database: stacdbs - Current Status: online - Status Message: 
Database: system - Current Status: online - Status Message: 


## Deleting Specific Databases in Neo4j

In [ ]:
databases_to_delete = [ 'stacdbs']

try:
    driver = GraphDatabase.driver(uri, auth=(user, password))
    
    with driver.session() as session:
        for db_name in databases_to_delete:
            # Avoid trying to delete 'system' or 'neo4j' database which are system databases
            if db_name not in ['neo4j', 'system']:
                session.run(f"DROP DATABASE {db_name} IF EXISTS")
                print(f"Database {db_name} has been deleted.")
            else:
                print(f"Cannot delete system database: {db_name}")
    
except Exception as e:
    print(f"Error connecting to Neo4j: {e}")
finally:
    driver.close()

Database stacdbs has been deleted.


## Retrive the data and check

In [110]:


try:
    driver = GraphDatabase.driver(uri, auth=(user, password))
    
    with driver.session(database=db_name) as session:
        result = session.run("MATCH (n) RETURN n")
        
        print(f"Data from the database {db_name}:")
        for record in result:
            print(record["n"])  

except Exception as e:
    print(f"Error connecting to Neo4j: {e}")
finally:
    driver.close()

Data from the database STACDBs:
<Node element_id='4:2da65ef2-b6cc-412d-80ce-8775f1c5036c:0' labels=frozenset({'BASIC_INFORMATIONS'}) properties={'composite_type': 'P1M', 'temporal_extent_start': '2020-01-08T16:22:00Z', 'spatial_extent': [5.7635, 46.7937, 15.1528, 55.9456], 'description': 'Sentinel-2 L3A WASP Products of Germany processed by DLR using the WASP processor', 'instrument': 'MSI', 'title': 'Sentinel-2 L3A Monthly WASP Products', 'sensor_type': 'OPTICAL', 'platform_serial_identifier': 'A/B', 'platform': 'Sentinel-2', 'license': 'CC-BY-4.0', 'name': 'S2_L3A_WASP', 'orbit_type': 'LEO', 'doi': '10.15489/4hcq6dgkj648'}>
<Node element_id='4:2da65ef2-b6cc-412d-80ce-8775f1c5036c:1' labels=frozenset({'BASIC_INFORMATIONS'}) properties={'license': 'CC-BY-4.0', 'name': 'S2_L2A_MAJA', 'temporal_extent_start': '2020-01-08T16:22:00Z', 'spatial_extent': [5.7635, 46.7937, 15.1528, 55.9456], 'description': 'Sentinel-2 L2A Products of Germany processed by DLR using the MAJA processor', 'orbit_

In [127]:
%pip install langchain langchain-community neo4j openai google-generative-ai

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement google-generative-ai (from versions: none)
ERROR: No matching distribution found for google-generative-ai

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [137]:
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from neo4j import GraphDatabase

graph = Neo4jGraph(
    url=uri,
    username=user,
    password=password,
    database=db_name
)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) 


chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, allow_dangerous_requests=True )

question = " Is there a specific download link available for accessing the satellite data related to Frankfurt within the S2_L2A_MAJA dataset?"

response = chain.invoke({"query": question})

print("Response:", response)



> Entering new GraphCypherQAChain chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}